In [17]:
##just messing around with different indicators and trying to predict the percent change of the stock price
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import time
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import yfinance as yf
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.regularizers import l1_l2
##import mean absolute error
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanAbsolutePercentageError
import statsmodels.api as sm
import scipy.stats as stats
import os

def indicators(df):
    true_range = np.maximum(df['High'] - df['Low'], 
    np.maximum(np.abs(df['High'] - df['Close'].shift(1)), 
    np.abs(df['Close'].shift(1) - df['Low'])))
    df['ATR'] = true_range.rolling(window=14).mean()
    df['ATR%'] = (df['ATR'] / df['Close']) * 100

    df['200ma'] = df['Close'].rolling(window=200).mean()
    df['14ma'] = df['Close'].rolling(window=14).mean()
    df['14ma%'] = ((df['Close'] - df['14ma']) / df['14ma']) * 100
    df['200ma%'] = ((df['Close'] - df['200ma']) / df['200ma']) * 100
    df['14ma-200ma'] = df['14ma'] - df['200ma']
    df['atr'] = df['High'] - df['Low']
    df['14ma_crossover'] = np.where(df['Close'] > df['14ma'], 1, 0)
    df['200ma_crossover'] = np.where(df['Close'] > df['200ma'], 1, 0)
    df['200DAY_ATR'] = df['200ma'] + df['ATR']
    df['200DAY_ATR-'] = df['200ma'] - df['ATR']
    df['200DAY_ATR%'] = df['200DAY_ATR']/df['Close']
    df['200DAY_ATR-%'] = df['200DAY_ATR-']/df['Close']
    df['percent_from_high'] = ((df['Close'] - df['Close'].cummax()) / df['Close'].cummax()) * 100
    df['new_high'] = df['Close'] == df['Close'].cummax()
    df['days_since_high'] = (~df['new_high']).cumsum() - (~df['new_high']).cumsum().where(df['new_high']).ffill().fillna(0)
    df['percent_range'] = (df['High'] - df['Low'])/df['Close']*100
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    df['VWAP%'] = df['Close']/df['VWAP']
    df['VWAP%'] = df['VWAP%']/df['VWAP%'].shift(1)*100 - 100
    df['percent_change_Close'] = df['Close']/df['Close'].shift(1)*100 - 100
    df['percent_change_Close_std'] = df['percent_change_Close'].rolling(window=200).std()
    df['percent_change_Volume'] = df['Volume']/df['Volume'].shift(1)*100 - 100
    df['std_percent_change_Close'] = df['percent_change_Close'].rolling(window=14).std()
    df['std_percent_change_Close>1'] = np.where(df['std_percent_change_Close'] > 2, 1, 0)
    df['cumsum_percent_change_Close'] = df['percent_change_Close'].rolling(window=14).sum()
    df['14_day_range'] = (df['Close'].cummax() - df['Close'])/df['Close']*100
    df['Percent_ATR'] = (df['ATR'] / df['Close']) * 100
    df['Volatility_Ratio'] = (df['High'] - df['Low']) / df['ATR']
    df['ATR%_change'] = df['ATR%'].pct_change(periods=45)
    df['pct_change_std'] = df['percent_change_Close'].rolling(window=60).std()
    threshold_multiplier = 0.65
    df['abnormal_pct_change_threshold'] = df['percent_change_Close'].rolling(window=20).mean() + threshold_multiplier * df['pct_change_std']
    df['days_since_abnormal_pct_change'] = 0  # Initialize column
    last_abnormal_index = None  # To keep track of the last index when pct_change was abnormal
    for i in range(len(df)):
        if df.iloc[i]['percent_change_Close'] > df.iloc[i]['abnormal_pct_change_threshold']:
            last_abnormal_index = i
        if last_abnormal_index is not None:
            df.iloc[i, df.columns.get_loc('days_since_abnormal_pct_change')] = i - last_abnormal_index
    df.drop(['pct_change_std', 'abnormal_pct_change_threshold'], axis=1, inplace=True)
    significant_figures = [10, 100, 500, 1000]  # Add or remove as needed
    custom_values = [69]  # Add any specific custom values
    multiples = []
    for figure in significant_figures:
        multiple = figure
        while multiple <= df['Close'].max():
            multiples.append(multiple)
            multiple += figure
    psych_levels = set(multiples + custom_values)
    rolling_min = df['Close'].rolling(window=500).min()
    df['psych_level'] = 0
    for level in psych_levels:
        df['psych_level'] = np.where(abs(df['Close'] - level) / level <= 0.01, 1, df['psych_level'])
    df['psych_level'] = np.where(abs(df['Close'] - rolling_min) / rolling_min <= 0.01, 1, df['psych_level'])

    df['direction_flipper'] = 0
    for i in range(1, len(df)):
        if df.loc[df.index[i], 'percent_change_Close'] > 0:
            df.loc[df.index[i], 'direction_flipper'] = 1
        else:
            df.loc[df.index[i], 'direction_flipper'] = 0

    keltner_central = df['Close'].ewm(span=20).mean()
    keltner_range = df['ATR']*1.5  # Using ATR from earlier calculation
    df['KC_UPPER%'] = ((keltner_central + keltner_range) - df['Close']) / df['Close'] * 100
    df['KC_LOWER%'] = (df['Close'] - (keltner_central - keltner_range)) / df['Close'] * 100
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    fast_ema = df['Close'].ewm(span=12, adjust=False).mean()
    slow_ema = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = fast_ema - slow_ema
    df['PPO'] = ((df['MACD'] / df['Close'].rolling(window=14).mean()) * 100)
    log_returns = np.log(df['Close'] / df['Close'].shift(1))
    df['Volatility'] = log_returns.rolling(window=14).std() * np.sqrt(14)
    rolling_mean = df['Close'].rolling(window=20).mean()
    rolling_std = df['Close'].rolling(window=20).std()
    lower_band = rolling_mean - (rolling_std * 2)
    upper_band = rolling_mean + (rolling_std * 2)
    df['B%'] = (df['Close'] - lower_band) / (upper_band - lower_band)
    df['positive_streak'] = 0
    df['negative_streak'] = 0

    positive_streak, negative_streak = 0, 0
    for i in range(1, len(df)):
        change = df.loc[df.index[i], 'percent_change_Close']
        if change > 0:  # If the change is positive
            positive_streak += 1
            negative_streak = 0
        elif change < 0:  # If the change is negative
            negative_streak += 1
            positive_streak = 0
        else:  # If no change, reset both streaks
            positive_streak, negative_streak = 0, 0

        # Assign the streak counts to the DataFrame
        df.loc[df.index[i], 'positive_streak'] = positive_streak
        df.loc[df.index[i], 'negative_streak'] = negative_streak




    df = df.round(10)

    df = df.drop(['Open', 'High', 'Low', 'Close', 'Adj Close',"VWAP","200DAY_ATR-","200DAY_ATR", 'Volume', 'atr', '200ma', '14ma'], axis=1)

    return df


def merge_macro_data(best_fit_stock_data, macro_data_folder):
    MergeErrorCounter = 0

    best_fit_stock_data['Date'] = pd.to_datetime(best_fit_stock_data['Date'])
    best_fit_stock_data.sort_values('Date', inplace=True)

    date_variations = ['date', 'Date', 'DATE', 'time', 'Time', 'TIME', 'datetime', 'Datetime', 'DATETIME']

    for file in os.listdir(macro_data_folder):
        file_path = os.path.join(macro_data_folder, file)

        try:
            macro_data = pd.read_csv(file_path)
            date_col = next((col for col in macro_data.columns if col.strip().lower() in date_variations), None)
            if date_col is None:
                print(f"No date column found in {file_path}. Skipping.")
                MergeErrorCounter += 1
                continue

            if MergeErrorCounter > 0:
                print(f"Processing {file_path} with date column {date_col}.")  # Debugging print

            # Ensure the date column is in datetime format
            macro_data[date_col] = pd.to_datetime(macro_data[date_col])
            macro_data.sort_values(date_col, inplace=True)

            # Rename columns in macro_data to ensure uniqueness, appending the base file name
            file_base = os.path.splitext(os.path.basename(file))[0]
            macro_data.columns = [f"{col}_{file_base}" if col != date_col else 'Date' for col in macro_data.columns]

            # Merge the macro data with the stock data using merge_asof()
            best_fit_stock_data = pd.merge_asof(
                best_fit_stock_data,
                macro_data,
                left_on='Date',
                right_on='Date',
                direction='nearest'
            )

        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            MergeErrorCounter += 1
            continue
    best_fit_stock_data.ffill(inplace=True)
    total_files = len(os.listdir(macro_data_folder))
    print(f"Successfully merged {100 - (MergeErrorCounter/total_files)*100:.2f}% of files.")
    best_fit_stock_data['Date'] = best_fit_stock_data['Date'].dt.strftime('%Y-%m-%d')
    return best_fit_stock_data


def get_stock_data(ticker, start_date, end_date=None):
    if end_date == None:
        end_date = datetime.datetime.today().strftime('%Y-%m-%d')
    if start_date == None:
        start_date = '2010-01-01'
    df = yf.download(ticker, start_date, end_date)
    df.index = pd.to_datetime(df.index)
    return df

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    y_true = y_true[non_zero_indices]
    y_pred = y_pred[non_zero_indices]
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def squash_col_outliers(
    df: pd.DataFrame, 
    col_name: None, 
    min_quantile: float =0.01, 
    max_quantile:float =0.99
): 
    
    if col_name is None:
        for col in df.columns:
            if df[col].dtype == 'float64':
                q_lo = df[col].quantile(min_quantile)
                q_hi  = df[col].quantile(max_quantile)

                df.loc[df[col] >= q_hi, col] = q_hi
                df.loc[df[col] <= q_lo, col] = q_lo
    return df



###LSTM prediction

def LSTM_prediction(df, target_col, sequence_length):
    if target_col not in df.columns:
        raise ValueError(f"The target column {target_col} is not in dataframe.")

    X = df.drop(columns=[target_col]).values  # Features
    y = df[target_col].values  # Target
    
    X_sequences, y_sequences = [], []
    for i in range(sequence_length, len(X)):
        X_sequences.append(X[i-sequence_length:i])
        y_sequences.append(y[i])
    X_sequences, y_sequences = np.array(X_sequences), np.array(y_sequences)
    
    X_sequences = X_sequences.astype('float16')
    y_sequences = y_sequences.astype('float16')

    X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.1, random_state=3301)
    
    print(f"X_train shape: {X_train.shape[2]}")

    model = Sequential([
        LSTM(X_train.shape[2]*2, return_sequences=True, input_shape=(sequence_length, X_train.shape[2])),
        Dropout(0.1),
        LSTM(X_train.shape[2], return_sequences=False),
        Dropout(0.1),
        Dense(round(X_train.shape[2]/2), activation='leaky_relu'),
        Dropout(0.1),
        Dense(1)
    ])

    learning_rate = 0.70
    batch_size = 512
    clipnorm = 0.85

    model.compile(optimizer=Adam(learning_rate, clipnorm=clipnorm), 
                  loss='mean_absolute_percentage_error',  # Using built-in MAPE as loss
                  metrics=[MeanAbsolutePercentageError()])  # Using MAPE as a metric

    reduce_lr_callback = ReduceLROnPlateau(monitor='mean_absolute_percentage_error', factor=0.75, patience=10, min_lr=1e-6, restore_best_weights=True, verbose=1)
    early_stopping_callback = EarlyStopping(monitor='mean_absolute_percentage_error', patience=15, restore_best_weights=True, verbose=1)

    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[reduce_lr_callback, early_stopping_callback]
    )

    predictions = model.predict(X_test)
    lowest_mape = min(history.history['val_mean_absolute_percentage_error'])
    print(f"Lowest Mean Absolute Percentage Error (MAPE) achieved: {lowest_mape:.2f}%")

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history['mean_absolute_percentage_error'], name='Training MAPE'))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_mean_absolute_percentage_error'], name='Validation MAPE'))

    x = history.epoch
    y = history.history['val_mean_absolute_percentage_error']
    z = np.polyfit(x, y, 3)
    p = np.poly1d(z)
    fig.add_trace(go.Scatter(x=x, y=p(x), name='Validation MAPE Trendline', line=dict(color='black', width=2, dash='dash')))

    
    fig.update_layout(title='Training and Validation MAPE')
    ##fig.show()
    return predictions



def PlotData(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['14ma%'], name='14 Day Moving Average %'))
    fig.add_trace(go.Scatter(x=df.index, y=df['200DAY_ATR%'], name='200 Day Moving Average Plus ATR %'))
    fig.add_trace(go.Scatter(x=df.index, y=df['200DAY_ATR-%'], name='200 Day Moving Average Minus ATR %'))
    fig.update_layout(title='AAPL Close Price History')
    ##fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['percent_range'], name='percent_range'))
    fig.add_trace(go.Scatter(x=df.index, y=df['percent_from_high'], name='percent_from_high'))
    fig.add_trace(go.Scatter(x=df.index, y=df['percent_change_Close'], name='percent_change_Close'))
    fig.add_trace(go.Scatter(x=df.index, y=df['VWAP%'], name='VWAP%'))
    ##fig.show()



    color_scale = [
        [0.0, "green"],   # start with red
        [0.5, "yellow"],  # middle with yellow
        [1.0, "red"]  # end with green
    ]

    difference = np.abs(df['percent_change_Close'] - df['Predictions'])
    norm_difference = (difference - difference.min()) / (difference.max() - difference.min())

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index, y=df['percent_change_Close'], name='percent_change_Close', line=dict(color='green', width=2)))

    fig.add_trace(go.Scatter(x=df.index, y=df['Predictions'], name='Predictions', line=dict(color='blue', width=2)))

    fig.add_trace(go.Scatter(x=df.index, y=df['percent_change_Close'] - df['Predictions'],
                             name='percent_change_Close - Predictions',
                             mode='lines+markers', # This includes both lines and markers
                             line=dict(color='gray', width=0.5), # Default line connecting markers
                             marker=dict(size=6, color=norm_difference, colorscale=color_scale, showscale=False)))

    fig.update_layout(showlegend=True, title='AAPL Close Price History')
    fig.update_xaxes(title_text='Date')

    fig.update_layout(title='AAPL Close Price History')
    
    fig.show()

    df['Prediction_Error'] = df['percent_change_Close'] - df['Predictions']

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=df['Prediction_Error'], name='Prediction Error', nbinsx=50))
    mean_error = np.mean(df['Prediction_Error'])
    fig.add_vline(x=mean_error, line_width=2, line_dash="dash", line_color="red")
    fig.update_layout(
        title="Prediction Error Histogram",
        xaxis_title="Prediction Error",
        yaxis_title="Count",
        bargap=0.05,  # Smaller gap between bars for smoother histogram
    )

    ##fig.show()

def scale_data(df):
    df.ffill(inplace=True)
    df.bfill(inplace=True)

    scalers = {}
    df_scaled = pd.DataFrame(index=df.index) 
    for col in df.columns:

        if col == 'Date':
            print(f"Skipping {col} column.")
            continue

        if not pd.api.types.is_numeric_dtype(df[col]):
            
            continue



        if df[col].dtype == 'bool':
            df[col] = df[col].astype(int)
            scaler = MinMaxScaler()
        else:
            df[col] = df[col] + 0.00001  # Add a small amount to avoid division by zero
            scaler = RobustScaler()

        df_scaled[col] = scaler.fit_transform(df[[col]])  # Scale each column individually
        scalers[f'scaler_{col}'] = scaler  # Store each scaler with a unique name
        
    return df_scaled, scalers


def main():
    df = get_stock_data('AAPL', '1990-01-01')
    df = indicators(df)
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    df = merge_macro_data(df, "Data/MacroData")
    df = squash_col_outliers(df, col_name=None, min_quantile=0.01, max_quantile=0.99)
    df = df.iloc[200:]


    df_scaled, scalers = scale_data(df)  # Adjusted scale_data function

    target_col = 'percent_change_Close'
    predictions = LSTM_prediction(df_scaled, target_col, sequence_length=45)

    predictions = predictions.reshape(-1, 1)
    predictions = scalers[f'scaler_{target_col}'].inverse_transform(predictions)
    prediction_length = len(predictions)
    df_scaled = df_scaled.iloc[-prediction_length:]  # Align df to match prediction length
    df_scaled['Predictions'] = predictions

    for col in df_scaled.columns:
        if col != 'Predictions':
            if col != target_col:
                if col != 'Date':
                    df_scaled[col] = scalers[f'scaler_{col}'].inverse_transform(df_scaled[[col]])

    PlotData(df_scaled)
    df_scaled['Prediction_Error'] = df_scaled['percent_change_Close'] - df_scaled['Predictions']
    df_scaled['Prediction_Error'] = df_scaled['Prediction_Error'].abs()
    df_scaled['Prediction_Error'] = df_scaled['Prediction_Error']/df_scaled['percent_change_Close']
    df_scaled['Prediction_Error'] = df_scaled['Prediction_Error'].abs()
    df_scaled['Prediction_Error'] = df_scaled['Prediction_Error'].round(2)

    print(target_col)
    print(df_scaled['percent_change_Close'].describe())

    print("Prediction_Error")
    print(df_scaled['Prediction_Error'].describe())


    df_scaled.to_csv('Data/Predictions/Percentage_Indicators.csv')

if __name__ == '__main__':
    main()

[*********************100%%**********************]  1 of 1 completed
Successfully merged 100.00% of files.
Skipping Date column.
X_train shape: 46
Epoch 1/100
15/15 [==============================] - 2s 35ms/step - loss: 62412.6562 - mean_absolute_percentage_error: 62412.6562 - val_loss: 11546.0986 - val_mean_absolute_percentage_error: 11546.0986 - lr: 0.7000
Epoch 2/100
15/15 [==============================] - 0s 14ms/step - loss: 14041.2510 - mean_absolute_percentage_error: 14041.2510 - val_loss: 5742.3184 - val_mean_absolute_percentage_error: 5742.3184 - lr: 0.7000
Epoch 3/100
15/15 [==============================] - 0s 13ms/step - loss: 15580.5498 - mean_absolute_percentage_error: 15580.5498 - val_loss: 8209.6650 - val_mean_absolute_percentage_error: 8209.6650 - lr: 0.7000
Epoch 4/100
15/15 [==============================] - 0s 14ms/step - loss: 22027.2656 - mean_absolute_percentage_error: 22027.2656 - val_loss: 9633.5469 - val_mean_absolute_percentage_error: 9633.5469 - lr: 0.7000

percent_change_Close
count    833.000000
mean       0.009305
std        0.689977
min       -2.184885
25%       -0.365207
50%        0.011761
75%        0.441803
max        2.317286
Name: percent_change_Close, dtype: float64
Prediction_Error
count      833.000000
mean        74.209736
std        974.804130
min          0.010000
25%          2.950000
50%          7.610000
75%         20.220000
max      27080.850000
Name: Prediction_Error, dtype: float64
